<a href="https://colab.research.google.com/github/saied-ahemd/spam_detector/blob/main/spamFilteringDeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [141]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.metrics import confusion_matrix, accuracy_score
import pickle
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('spam.csv', encoding='ISO-8859-1')


In [ ]:
# drop useless data 
df = df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)

In [ ]:
# rename the coulmn
df.columns = ['label','data']

In [ ]:
# now let's make the ham = 0 and spam = 1
df['label']= df['label'].map({'ham':0,'spam':1})

In [ ]:
# now we will get the value from the data
X = df['data'].values
y = df['label'].values

In [ ]:
# now we will spilt our data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
# now let's vectorize our data
# first thing we will tokinize or data
token = Tokenizer()
token.fit_on_texts(X_train)
print(X_train[1])

In [ ]:
# applying sequences from the dictionary on both training and test dataset
encoded_train = token.texts_to_sequences(X_train)
encoded_test = token.texts_to_sequences(X_test)

In [ ]:
# now let's do a padding for out text because deep learning expexet the squences to be all the same len
max_len = 15
padded_train = pad_sequences(encoded_train,maxlen=max_len,padding='post')
padded_test = pad_sequences(encoded_test,maxlen=max_len,padding='post')

In [ ]:
# now we can build our model
vocab_size = len(token.word_index) + 1
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size,28,input_length=max_len),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(500,activation='tanh'),
                             tf.keras.layers.Dense(200,activation='relu'),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Dense(100,activation='relu'),
                             tf.keras.layers.Dense(1,activation='sigmoid'),

])

In [ ]:
# now let's compile our model
model.compile(loss='binary_crossentropy', metrics=['acc'])

In [ ]:
# now let's train our model
model.fit(x=padded_train,y=y_train,epochs=49)

In [110]:
# now let's see the eval
preds = (model.predict(padded_test) > 0.5).astype("int32")
accuracy_score(y_test, preds)

0.9811659192825112

In [90]:
# now let's save the model
model.save('spamFilteringModel/my_model')

INFO:tensorflow:Assets written to: spamFilteringModel/my_model/assets


In [ ]:
# now let's save our tokanizer 
with open('spamFilteringModel/tokenizer.pkl', 'wb') as output:
   pickle.dump(token, output, pickle.HIGHEST_PROTOCOL)

In [ ]:
# now let's load our model
with open('spamFilteringModel/tokenizer.pkl', 'rb') as input:
    tokenizer = pickle.load(input)

In [79]:
# now let's create a function to process the input prediction
def Pro_pre(tex):
  text = token.texts_to_sequences(tex)
  text = pad_sequences(text,maxlen=max_len,padding='post')
  return text
  

In [139]:
tex = ["Hi i'm your sister call me after work"]
t = Pro_pre(tex)

In [91]:
# now let's call the model
new_model = tf.keras.models.load_model('spamFilteringModel/my_model')
